## Imports

In [ ]:
# Conv2D is spatial convolution over images
# MaxPooling2D is a max pooling operation for 2D spatial data
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
# sequential is good for when we have a single dataset and looking for single output
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.applications.resnet50 import ResNet50
import imghdr
import cv2
import tensorflow as tf
import os
import random
import numpy as np
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import img_to_array, load_img
from keras.models import Model

from skimage import data, color, img_as_ubyte
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
from IPython.display import display

## Data collection and gpu configuration

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
     

# removing dodgy images
# open computer vision
# allows us to check extensions of images

# variable to data directory
data_dir = 'tonsildb'

# extensions of images
img_exts = ['jpeg', 'jpg', 'bmp', 'png']

data = tf.keras.utils.image_dataset_from_directory('tonsildb', seed=42)
images_np = np.concatenate([x for x, y in data], axis=0)
labels_np = np.concatenate([y for x, y in data], axis=0)

## Random configuration

In [ ]:
def set_seed(seed: int = 42) -> None:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    tf.keras.utils.set_random_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)


set_seed()

## Available pre processing pipeline functions

In [ ]:
def color_correct(image):
    image = tf.image.adjust_contrast(image, contrast_factor=1.5)
    image = tf.image.adjust_brightness(image, delta=0.2)
    image = tf.clip_by_value(image, 0, 255)
    # image = tf.cast(image, tf.float32)/255.0
    return image


def rgb_to_ycbcr(rgb_image):
    R, G, B = tf.split(rgb_image, num_or_size_splits=3, axis=-1)
    Y = 0.299 * R + 0.587 * G + 0.114 * B
    Cb = -0.169 * R - 0.331 * G + 0.5 * B + 128
    Cr = 0.5 * R - 0.419 * G - 0.081 * B + 128
    ycbcr_image = tf.stack([Y, Cb, Cr], axis=3)
    ycbcr_image = tf.squeeze(ycbcr_image, axis=-1)
    return ycbcr_image


def color_mask_ycbcr(ycbcr_image):
    y, cb, cr = tf.split(ycbcr_image, num_or_size_splits=3, axis=-1)
    y_min = 92
    y_max = 145
    cb_min = 112
    cb_max = 142
    cr_min = 135
    cr_max = 185
    y_mask = tf.math.logical_and(y >= y_min, y <= y_max)
    cb_mask = tf.math.logical_and(cb >= cb_min, cb <= cb_max)
    cr_mask = tf.math.logical_and(cr >= cr_min, cr <= cr_max)
    color_mask = tf.math.logical_and(
        tf.math.logical_and(y_mask, cb_mask), cr_mask)
    color_mask = tf.cast(color_mask, dtype=tf.float32)
    masked_image = ycbcr_image * color_mask
    # masked_image = tf.cast(masked_image, tf.float32)/255.0
    return masked_image

def color_mask_rgb(rgb_image):
    #print_rgb_channel_values(rgb_image)
    r, g, b = tf.split(rgb_image, num_or_size_splits=3, axis=-1)
    r_min = 0
    r_max = 320
    g_min = 0
    g_max = 320
    b_min = 0
    b_max = 320
    r_mask = tf.math.logical_and(r >= r_min, r <= r_max)
    g_mask = tf.math.logical_and(g >= g_min, g <= g_max)
    b_mask = tf.math.logical_and(b >= b_min, b <= b_max)
    color_mask = tf.math.logical_and(
        tf.math.logical_and(r_mask, g_mask), b_mask)
    color_mask = tf.cast(color_mask, dtype=tf.float32)
    masked_image = rgb_image * color_mask
    # masked_image = tf.cast(masked_image, tf.float32)/255.0
    return masked_image


## Pipeline

In [ ]:
def pipeline_for_individial_image(image):
    image = color_correct(image=image)
    image = rgb_to_ycbcr(rgb_image=image)
    return image

def pipeline(data):
    data = data.map(lambda x, y: (color_correct(x), y))
    #data = data.map(lambda x, y: (color_mask_rgb(x), y))
    #data = data.map(lambda x, y: (rgb_to_ycbcr(x), y))
    #data = data.map(lambda x, y: (color_mask_ycbcr(x), y))
    return data

data = pipeline(data)

## See what color correct does

In [ ]:
color_corrected_data = data.map(lambda x, y: (color_correct(x), x, y))
for color_corrected_batch, original_batch, label_batch in color_corrected_data.take(1):
    for i in range(32):
        plt.subplot(4, 16, i*2 + 1)
        plt.imshow(original_batch[i]/255.0)
        plt.axis('off')
        plt.subplot(4, 16, i*2 + 2)
        plt.imshow(color_corrected_batch[i]/255.0)
        plt.axis('off')
    plt.show()

## See what rgb color mask does

In [ ]:
data_masked = data.map(lambda x, y: (color_mask_rgb(x), x, y))
for color_corrected_batch, original_batch, label_batch in data_masked.take(1):
    for i in range(32):
        plt.subplot(4, 16, i*2 + 1)
        plt.imshow(original_batch[i]/255.0)
        plt.axis('off')
        plt.subplot(4, 16, i*2 + 2)
        plt.imshow(color_corrected_batch[i]/255.0)
        plt.axis('off')
    plt.show()

## See what rgb to ycbcr does

In [ ]:
data_ycbcr = data.map(lambda x, y: (rgb_to_ycbcr(x), x, y))
for color_corrected_batch, original_batch, label_batch in data_ycbcr.take(1):
    for i in range(32):
        plt.subplot(4, 16, i*2 + 1)
        plt.imshow(original_batch[i]/255.0)
        plt.axis('off')
        plt.subplot(4, 16, i*2 + 2)
        plt.imshow(color_corrected_batch[i]/255.0)
        plt.axis('off')
    plt.show()

## See what ycbcr color mask does

In [ ]:
data_masked = data.map(lambda x, y: (color_mask_ycbcr(x), x, y))
for color_corrected_batch, original_batch, label_batch in data_masked.take(1):
    for i in range(32):
        plt.subplot(4, 16, i*2 + 1)
        plt.imshow(original_batch[i]/255.0)
        plt.axis('off')
        plt.subplot(4, 16, i*2 + 2)
        plt.imshow(color_corrected_batch[i]/255.0)
        plt.axis('off')
    plt.show()

## See what green color mask does

In [ ]:
data_masked = data.map(lambda x, y: (apply_sobel_filter(x), x, y))
for color_corrected_batch, original_batch, label_batch in data_masked.take(1):
    for i in range(32):
        plt.subplot(4, 16, i*2 + 1)
        plt.imshow(original_batch[i]/255.0)
        plt.axis('off')
        plt.subplot(4, 16, i*2 + 2)
        plt.imshow(color_corrected_batch[i]/255.0)
        plt.axis('off')
    plt.show()

## OpenCV pipeline functions

In [ ]:
def print_image(image_np):
    plt.imshow(image_np/255.0)
    plt.show()

def print_image_grayscale(image_np, gray = False):
    if gray == True:
        plt.imshow(image_np/255.0, cmap='gray')
    else:
        plt.imshow(image_np)
    plt.show()

def combine_mask_with_image(mask, image_np):
    image_np = cv2.bitwise_and(image_np, image_np, mask = mask)
    return image_np


def apply_threshold_rgb(images_np, lower = np.array([0,0,0]), upper = np.array([255,255,255])):
    binary_np = cv2.inRange(images_np, lower, upper)
    return binary_np


def apply_threshold_grayscale(grayscale_np, lower=0, upper=255):
    binary_np = cv2.inRange(grayscale_np, lower, upper)
    return binary_np

def convert_to_grayscale(images_np):
    #gray_np = np.dot(images_np[..., :3], [0.2989, 0.5870, 0.1140])
    gray_np = cv2.cvtColor(images_np, cv2.COLOR_BGR2GRAY)
    return gray_np

def convert_rgb_to_ycbcr(image_np):
    image_np = image_np.astype(np.uint8)
    ycbcr_image = cv2.cvtColor(image_np, cv2.COLOR_RGB2YCrCb)
    return ycbcr_image
'''
def transform_using_watershed(grad_mag_np, image_np, labels_np):
    grad_mag_np = cv2.convertScaleAbs(grad_mag_np)
    ret, thresh = cv2.threshold(grad_mag_np, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel = np.ones((3,3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7*dist_transform.max(), 255, 0)
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    ret, markers = cv2.connectedComponents(sure_fg)
    
    
    markers = markers + 1
    markers[unknown == 255] = 0
    markers = markers.astype('int32')
    image_np = cv2.convertScaleAbs(image_np)
    markers = cv2.watershed(image_np, markers)
    img_output = np.zeros_like(image_np)
    img_output[markers == -1] = 255
    plt.imshow(img_output)
    plt.show()
'''

def tensor_dataset_from_np(images_np, labels_np):
    return tf.data.Dataset.from_tensor_slices((images_np,labels_np)).shuffle(buffer_size=1000).batch(batch_size=32)

## Image segmentation functions

In [ ]:
def neighbourhood_region_labelling(np_binary_image):
    #labels is a numpy array that contains the labelled image. Each pixel is assigned a label value corresponding to the inex of the connected component that pixel belongs to
    #num_labels is an integer representing number of connected components
    num_labels, labels = cv2.connectedComponents(np_binary_image, connectivity=8)
    return labels

## Edge detection and smoothing functions

In [ ]:
def apply_gaussian_blur(images_np):
    # second argument is kernel size. Larger the kernel size means wider distribution of weights
    # third argument is sigma - larger the sigma the more spread out the blur effect. Pixels far away have greater influence on final output
    # A larger kernel size can help to smooth out larger features in the image, while a larger sigma value can help to remove finer details and noise
    blurred_image_np = cv2.GaussianBlur(images_np, (7, 7), 15)
    return blurred_image_np

def calculate_gradient_mag_using_sobel(grayscale_np):
    #Second argument is the data type of the output gradient image
    #Third - Order of derivative in x direction
    #Fourth - Order of derivative in y direction
    #Fifth - Sobel Kernel to be used in convolution
    grad_x = cv2.Sobel(grayscale_np, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(grayscale_np, cv2.CV_64F, 0, 1, ksize=3)
    grad_mag = np.sqrt(grad_x**2 + grad_y**2)
    return grad_mag

def calculate_gradient_mag_using_canny(grayscale_np, lower_thresh = 0, upper_thresh = 0):
    #second argument - grad mag threshold. Any edge with grad mag lower than this is discarded as weak edges
    #third argument - grad mag threshold but any edge with grad mag higher considered as strong edges
    grayscale_np = grayscale_np.astype(np.uint8)
    edges_np = cv2.Canny(grayscale_np,lower_thresh, upper_thresh)
    return edges_np


def get_contours_from_binary_edges(edges):
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    return contours

## Vision modelling and detecting shapes

In [ ]:

def get_circles_from_edges(edges, min_radius = 0, max_radius = 100):
    #first argument is the binary image of edges
    #Second argument is variation of hough transform -
    #variants include cv2.HOUGH_STANDARD, cv2.HOUGH_PROBABALISTIC, cv2.HOUGH_GRADIENT
    #third argument is resolution of accumulator array - represents ratio between image resolution and resolution of accumulator array - determines granularity of search - smaller leads to more finer search
    #fourth argument is the minimum distance between centers of detected circles
    #param1 is the higher threshold for the canny edge detector
    #fourth argument is Min and max radius of circles found
    circles = cv2.HoughCircles(edges, cv2.HOUGH_GRADIENT, 1,20, param1=50, param2= 30, minRadius=min_radius, maxRadius=max_radius)
    
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        distances = np.sqrt((circles[:, 0] - edges.shape[1]/2)**2 + (circles[:, 1] - edges.shape[0]/2)**2)
        sorted_circles = circles[np.argsort(distances)]
        sorted_circles = np.expand_dims(sorted_circles, axis=0)
        return sorted_circles
    
    return circles


def draw_circles_on_image(image_np, circles):
    if circles is not None:
        circles = np.uint16(np.around(circles))
        #because of break after this, it will only take the top most circle i.e. closest to centre
        for i in circles[0, :]:
            center = (i[0], i[1])
            # circle center
            cv2.circle(image_np, center, 1, (0, 100, 100), 3)
            # circle outline
            radius = i[2]
            cv2.circle(image_np, center, radius, (255, 0, 255), 3)
            break
    return image_np

def create_and_apply_circle_mask(image_np, circles):
    if circles is not None:
        circle = circles[0][0]
        mask = np.zeros(image_np.shape[:2], dtype=np.uint8)
        cv2.circle(mask, (circle[0], circle[1]), circle[2], (255, 255, 255), -1)
        result = cv2.bitwise_and(image_np, image_np, mask=mask)
    else:
        result = np.zeros_like(image_np)
    return result

## Pipeline

In [ ]:
def pipeline(images_np, labels_np):
    images_np, labels_np = apply_gaussian_blur(images_np, labels_np)
    data_from_np = tensor_dataset_from_np(images_np, labels_np)
    
for i in range(len(images_np)):
    print_image(images_np[i])
    #binary_np= apply_threshold_rgb(images_np[i], np.array([0,0,0]), np.array([250,250,250]))
    #image_np = combine_mask_with_image(binary_np, images_np[i])
    

    ycbcr_image = convert_rgb_to_ycbcr(images_np[i])
    binary_np = apply_threshold_rgb(ycbcr_image, np.array([22,130,110]), np.array([122,155,130]))
    image_np = combine_mask_with_image(binary_np, images_np[i])
    #print_image(image_np)
    
    
    blurred_np= apply_gaussian_blur(images_np[i])
    #print_image(blurred_np)
    gray_np = convert_to_grayscale(blurred_np)
    #print_image_grayscale(gray_np)
    #grad_mag_np = calculate_gradient_mag_using_sobel(gray_np)
    #print_image_grayscale(grad_mag_np, True)
    grad_mag_np = calculate_gradient_mag_using_canny(gray_np, 20, 25)
    #grad_mag_np = canny_skimage(gray_np)
    print_image_grayscale(grad_mag_np, True)
    circles = get_circles_from_edges(grad_mag_np, 50, 100)
    to_draw_np = images_np[i]
    masked_np = create_and_apply_circle_mask(images_np[i],circles)
    print_image(masked_np)
    #circles_np = draw_circle_on_image(to_draw_np, circles)
    #circles_np = draw_circles_on_image(to_draw_np, circles)
    #print_image(circles_np)
    #perform_hough_transform(grad_mag_np, images_np[i])
    #transform_using_watershed(grad_mag_np, images_np[i], labels_np[i])

## Actually preprocessing data

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=10,
    horizontal_flip=True,
    rescale=1./255
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

train_size = int(len(data) * .7)
val_size = int(len(data) * .2)
# +1 to make sure total is 12 as that was total no. of batches
test_size = int(len(data) * .1) + 1

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)
print(len(train))
print(len(val))
print(len(test))

In [ ]:
def return_np_array_from_dataset(ds):
    images_list = []
    labels_list = []
    for images, labels in ds.unbatch():
        images_list.append(images.numpy())
        labels_list.append(labels.numpy())
    images_np = np.array(images_list)
    labels_np = np.array(labels_list)
    return images_np, labels_np


train_images_np, train_labels_np = return_np_array_from_dataset(train)
print(len(train_images_np), ", ", len(train_labels_np))
val_images_np, val_labels_np = return_np_array_from_dataset(val)
print(len(val_images_np), ", ", len(val_labels_np))
test_images_np, test_labels_np = return_np_array_from_dataset(test)
print(len(test_images_np), ", ", len(test_labels_np))

In [ ]:
train_generator = train_datagen.flow(
    train_images_np,
    train_labels_np,
    shuffle=False,
    batch_size=32
)
val_generator = val_datagen.flow(
    val_images_np,
    val_labels_np,
    shuffle=False,
    batch_size=32
)
test_generator = test_datagen.flow(
    test_images_np,
    test_labels_np,
    shuffle=False,
    batch_size=32
)

## View images and labels generated

In [ ]:
#Change the generator in the next to choose which generator
images_to_show, labels_to_show = next(test_generator)
fig, axes = plt.subplots(nrows=2, ncols=32//2)
for i, ax in enumerate(axes.flatten()):
    print(i, " ", ax)
    ax.imshow(images_to_show[i])
    ax.set_title(f"{labels_to_show[i]}")
    ax.axis('off')
plt.tight_layout()
plt.show()

## Building the deep learning model

In [ ]:
'''
model = ResNet50(include_top=False, weights='imagenet',
                 input_shape=(256, 256, 3), classes=2, pooling='avg')
resnet_model = Sequential()
resnet_model.add(model)
for layer in resnet_model.layers:
    layer.trainable = False
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(1, activation='sigmoid'))
resnet_model.compile('adam', loss=tf.losses.BinaryCrossentropy(),
                     metrics=['accuracy'])
resnet_model.summary()
'''


model = Sequential()
# model.add(resnet_model)
# Conv2D(No. of filters, dimensions of filter, activation function, expected image size(only first time))
model.add(Conv2D(32, (3, 3), 1, activation='relu',
          padding="same", input_shape=(256, 256, 3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3, 3), 1, activation='relu'))
model.add(MaxPooling2D())


model.add(Conv2D(32, (3, 3), 1, activation='relu'))

model.add(Conv2D(32, (3, 3), 1, activation='relu', padding="same"))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile('adam', loss=tf.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
# model.summary()

## Fitting and testing the model

In [ ]:
hist = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    verbose=1
)

### Graph for error and accuracy

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
plt.plot(hist.history['accuracy'], color='green', label='accuracy')
plt.plot(hist.history['val_accuracy'],
         color='black', label='val_accuracy')
fig.suptitle('Loss & accuracy')
plt.legend(loc='upper left')
plt.show()

## Evaluation on test set

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

for i in range(2):
    x, y = test_generator.next()
    y_pred = model.predict(x)
    y_hat = []
    for x in y_pred:
        y_hat.append(x[0])
    pre.update_state(y, y_hat)
    re.update_state(y, y_hat)
    acc.update_state(y, y_hat)

print("Precision: ", pre.result().numpy())
print("Recall: ", re.result().numpy())
print("Accuracy: ", acc.result().numpy())
del model

## Saving the model

In [ ]:
model.save(os.path.join('models', 'tonsil_detector.h5'))